# Improved Strategic Visualizations

This notebook implements improved visualizations based on UX feedback:
- Faceted plots by position for cleaner comparison
- Discrete urgency bins instead of continuous color scale
- Reduced visual clutter with simpler markers
- Interactive hover tooltips
- Better handling of overlapping points

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.colors as pc
import os
import sys
from pathlib import Path

# Add parent directory to path for imports
sys.path.append(str(Path('../').resolve()))

def load_enhanced_data():
    """Load enhanced draft data with robust error handling and validation"""
    
    try:
        # Method 1: Try to import functions and load data from main notebook
        print("🔄 Attempting to load data from main analysis...")
        
        # Look for exported enhanced data CSV first
        enhanced_data_path = '../data/enhanced_draft_data.csv'
        if os.path.exists(enhanced_data_path):
            print(f"✅ Found exported data at {enhanced_data_path}")
            df = pd.read_csv(enhanced_data_path)
            return validate_and_prepare_data(df)
            
        # Method 2: Try to load from individual data files and reconstruct
        print("🔄 Attempting to reconstruct data from source files...")
        
        # Load base ranking data
        espn_path = '../data/espn_projections_20250814.csv'
        adp_path = '../data/fantasypros_adp_20250815.csv'
        vbd_path = '../draft_cheat_sheet.csv'
        
        if not all(os.path.exists(p) for p in [espn_path, adp_path, vbd_path]):
            raise FileNotFoundError("Required data files not found")
            
        # Load and merge data (simplified version)
        espn_df = pd.read_csv(espn_path)
        adp_df = pd.read_csv(adp_path) 
        vbd_df = pd.read_csv(vbd_path)
        
        # Basic merge and calculation (fallback)
        df = create_basic_enhanced_data(espn_df, adp_df, vbd_df)
        return validate_and_prepare_data(df)
        
    except Exception as e:
        print(f"❌ Error loading data: {str(e)}")
        print("\n🚨 SETUP REQUIRED:")
        print("1. Run the main espn_probability_matrix.ipynb notebook first")
        print("2. Or ensure these files exist:")
        print("   - ../data/espn_projections_20250814.csv")
        print("   - ../data/fantasypros_adp_20250815.csv") 
        print("   - ../draft_cheat_sheet.csv")
        print("3. Or export enhanced data as 'enhanced_draft_data.csv'")
        raise

def validate_and_prepare_data(df):
    """Validate required columns and data quality"""
    
    required_columns = [
        'player_name', 'position', 'espn_rank', 'adp_rank',
        'vbd_score', 'probability_available'
    ]
    
    # Check for required columns
    missing_cols = [col for col in required_columns if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    # Validate data types and ranges
    if df.empty:
        raise ValueError("Data is empty")
        
    if df['probability_available'].isna().any():
        print("⚠️ Warning: Found NaN values in probability_available, filling with 0.5")
        df['probability_available'] = df['probability_available'].fillna(0.5)
        
    # Ensure probability is between 0 and 1
    df['probability_available'] = df['probability_available'].clip(0, 1)
    
    # Create derived columns if missing
    if 'abs_divergence' not in df.columns:
        df['abs_divergence'] = abs(df['espn_rank'] - df['adp_rank'])
        
    if 'rank_divergence' not in df.columns:
        df['rank_divergence'] = df['espn_rank'] - df['adp_rank']
        
    if 'urgency_rank' not in df.columns:
        # Simple urgency calculation: lower probability = higher urgency
        df['urgency_rank'] = df['probability_available'].rank(method='min')
    
    print(f"✅ Data validated: {len(df)} players across {df['position'].nunique()} positions")
    return df

def create_basic_enhanced_data(espn_df, adp_df, vbd_df):
    """Create basic enhanced dataset when main analysis isn't available"""
    
    print("🔧 Creating basic enhanced dataset...")
    
    # Simple merge strategy (adjust column names as needed)
    # This is a fallback - the main notebook has more sophisticated logic
    
    try:
        # Merge ESPN and ADP data
        if 'player_name' in espn_df.columns and 'player_name' in adp_df.columns:
            merged_df = pd.merge(espn_df, adp_df, on='player_name', how='outer', suffixes=('_espn', '_adp'))
        else:
            raise ValueError("Cannot merge - player_name column not found")
            
        # Add VBD data if available
        if 'player_name' in vbd_df.columns:
            merged_df = pd.merge(merged_df, vbd_df[['player_name', 'vbd_score']], 
                               on='player_name', how='left')
        
        # Create basic probability calculation (simplified)
        merged_df['probability_available'] = 0.5  # Default fallback
        
        # Clean up column names
        column_mapping = {
            'rank_espn': 'espn_rank',
            'rank_adp': 'adp_rank',
            'position_espn': 'position',
            'position_adp': 'position'
        }
        
        for old, new in column_mapping.items():
            if old in merged_df.columns:
                merged_df[new] = merged_df[old]
                
        # Fill missing VBD scores
        if 'vbd_score' not in merged_df.columns:
            merged_df['vbd_score'] = 0
        merged_df['vbd_score'] = merged_df['vbd_score'].fillna(0)
        
        print(f"✅ Basic dataset created: {len(merged_df)} players")
        return merged_df
        
    except Exception as e:
        print(f"❌ Error creating basic dataset: {str(e)}")
        raise

# Load the enhanced data with error handling
try:
    enhanced_df = load_enhanced_data()
    print(f"\n📊 Successfully loaded {len(enhanced_df)} players")
    print(f"Positions: {', '.join(enhanced_df['position'].value_counts().index.tolist())}")
except Exception as e:
    print(f"\n❌ Failed to load data: {str(e)}")
    print("\nPlease run the setup steps mentioned above and try again.")
    raise

In [ ]:
# Data preparation - create discrete urgency bins
def prepare_visualization_data(df):
    """Prepare data with discrete urgency bins and flagged players"""
    
    try:
        # Validate input data
        if df is None or df.empty:
            raise ValueError("Input DataFrame is empty or None")
            
        # Check for required columns
        required_columns = ['probability_available', 'vbd_score']
        missing_cols = [col for col in required_columns if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing required columns: {missing_cols}")
        
        # Create copy to avoid modifying original
        viz_df = df.copy()
        
        # Create discrete urgency bins based on availability probability
        def categorize_urgency(availability):
            try:
                if pd.isna(availability):
                    return "Unknown"
                if availability > 0.8:
                    return "Safe"
                elif availability > 0.3:
                    return "Decision"
                else:
                    return "Urgent"
            except (TypeError, ValueError):
                return "Unknown"
        
        viz_df['urgency_bin'] = viz_df['probability_available'].apply(categorize_urgency)
        
        # Create abs_divergence if missing
        if 'abs_divergence' not in viz_df.columns:
            if 'espn_rank' in viz_df.columns and 'adp_rank' in viz_df.columns:
                viz_df['abs_divergence'] = abs(viz_df['espn_rank'] - viz_df['adp_rank'])
            else:
                viz_df['abs_divergence'] = 0  # Default if ranks missing
        
        # Flag players with large rank disagreements
        viz_df['is_flagged'] = viz_df['abs_divergence'] > 20
        
        # Normalize VBD scores for sizing (between 8-24 pixels)
        try:
            vbd_min, vbd_max = viz_df['vbd_score'].min(), viz_df['vbd_score'].max()
            if vbd_max > vbd_min:
                viz_df['marker_size'] = 8 + (viz_df['vbd_score'] - vbd_min) / (vbd_max - vbd_min) * 16
            else:
                viz_df['marker_size'] = 12  # Default size if all VBD scores are the same
        except (TypeError, ValueError):
            viz_df['marker_size'] = 12  # Default size if VBD calculation fails
        
        # Ensure marker sizes are reasonable
        viz_df['marker_size'] = viz_df['marker_size'].clip(8, 24)
        
        print(f"✅ Data preparation complete: {len(viz_df)} players")
        return viz_df
        
    except Exception as e:
        print(f"❌ Error preparing visualization data: {str(e)}")
        raise

# Prepare data with error handling
try:
    viz_data = prepare_visualization_data(enhanced_df)
    
    print(f"Data prepared: {len(viz_data)} players")
    print(f"Urgency distribution:")
    print(viz_data['urgency_bin'].value_counts())
    print(f"Flagged players (>20 rank divergence): {viz_data['is_flagged'].sum()}")
    
except Exception as e:
    print(f"❌ Failed to prepare visualization data: {str(e)}")
    print("Please check that the data was loaded correctly.")
    raise

In [ ]:
def create_faceted_strategic_plot(df, positions=['RB', 'WR', 'QB']):
    """Create faceted strategic plot by position with improved UX"""
    
    try:
        # Validate input data
        if df is None or df.empty:
            raise ValueError("Input DataFrame is empty or None")
            
        # Check for required columns
        required_columns = ['position', 'espn_rank', 'adp_rank', 'urgency_bin', 'marker_size']
        missing_cols = [col for col in required_columns if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing required columns for plotting: {missing_cols}")
        
        # Filter to main positions
        plot_df = df[df['position'].isin(positions)].copy()
        
        if plot_df.empty:
            raise ValueError(f"No data found for positions: {positions}")
        
        # Colorblind-friendly discrete colors
        color_map = {
            "Safe": "#2E8B57",      # Sea green
            "Decision": "#FF8C00",  # Dark orange  
            "Urgent": "#DC143C",    # Crimson
            "Unknown": "#808080"    # Gray for unknown urgency
        }
        
        # Create faceted scatter plot
        fig = px.scatter(
            plot_df,
            x='espn_rank',
            y='adp_rank', 
            facet_col='position',
            color='urgency_bin',
            color_discrete_map=color_map,
            size='marker_size',
            size_max=24,
            hover_data={
                'player_name': True,
                'vbd_score': ':.1f',
                'probability_available': ':.1%',
                'espn_rank': True,
                'adp_rank': True,
                'rank_divergence': ':.1f' if 'rank_divergence' in plot_df.columns else False,
                'marker_size': False,
                'urgency_bin': False
            },
            labels={
                'espn_rank': 'ESPN Rank (lower = better)',
                'adp_rank': 'ADP Rank (lower = better)',
                'player_name': 'Player',
                'vbd_score': 'VBD Score',
                'probability_available': 'Availability',
                'rank_divergence': 'Rank Divergence'
            },
            category_orders={'position': positions},
            height=600,
            width=1200
        )
        
        # Update traces for better opacity
        fig.update_traces(
            marker=dict(opacity=0.7, line=dict(width=0.5, color='white')),
            hovertemplate=(
                '<b>%{customdata[0]}</b><br>' +
                'ESPN Rank: %{x}<br>' +
                'ADP Rank: %{y}<br>' +
                'VBD Score: %{customdata[1]}<br>' +
                'Availability: %{customdata[2]}<br>' +
                ('Divergence: %{customdata[5]}<br>' if 'rank_divergence' in plot_df.columns else '') +
                '<extra></extra>'
            )
        )
        
        # Add perfect agreement line to each facet
        max_rank = min(200, max(plot_df['espn_rank'].max(), plot_df['adp_rank'].max()))
        for i, pos in enumerate(positions):
            fig.add_trace(
                go.Scatter(
                    x=[1, max_rank],
                    y=[1, max_rank],
                    mode='lines',
                    line=dict(dash='dash', color='gray', width=1),
                    name='Perfect Agreement',
                    showlegend=True if i == 0 else False,
                    hoverinfo='skip',
                    xaxis=f'x{i+1}' if i > 0 else 'x',
                    yaxis=f'y{i+1}' if i > 0 else 'y'
                )
            )
        
        # Highlight flagged players with gold rings (if flagged data exists)
        if 'is_flagged' in plot_df.columns:
            flagged_df = plot_df[plot_df['is_flagged']]
            if not flagged_df.empty:
                for i, pos in enumerate(positions):
                    pos_flagged = flagged_df[flagged_df['position'] == pos]
                    if not pos_flagged.empty:
                        fig.add_trace(
                            go.Scatter(
                                x=pos_flagged['espn_rank'],
                                y=pos_flagged['adp_rank'],
                                mode='markers',
                                marker=dict(
                                    size=pos_flagged['marker_size'] + 4,
                                    symbol='circle-open',
                                    line=dict(width=3, color='gold')
                                ),
                                name='High Divergence' if i == 0 else '',
                                showlegend=True if i == 0 else False,
                                hoverinfo='skip',
                                xaxis=f'x{i+1}' if i > 0 else 'x',
                                yaxis=f'y{i+1}' if i > 0 else 'y'
                            )
                        )
        
        # Update layout
        fig.update_layout(
            title={
                'text': 'Strategic Draft Analysis: ESPN vs ADP Rankings by Position',
                'x': 0.5,
                'font': {'size': 16}
            },
            plot_bgcolor='rgba(248,249,250,1)',
            paper_bgcolor='white',
            font=dict(size=12),
            legend=dict(
                orientation='h',
                yanchor='bottom',
                y=1.02,
                xanchor='center',
                x=0.5
            ),
            margin=dict(l=60, r=60, t=120, b=60)
        )
        
        # Ensure shared axes
        fig.update_xaxes(matches='x', range=[0, max_rank])
        fig.update_yaxes(matches='y', range=[0, max_rank])
        
        return fig
        
    except Exception as e:
        print(f"❌ Error creating faceted plot: {str(e)}")
        raise

# Create the improved faceted plot with error handling
try:
    faceted_plot = create_faceted_strategic_plot(viz_data)
    faceted_plot.show()

    print("\n🎯 IMPROVED Strategic Analysis:")
    print("🔴 URGENT: Crimson markers = High value + critical urgency (draft now)")
    print("🟠 DECISION: Orange markers = Value players in decision zone (consider drafting)")
    print("🟢 SAFE: Green markers = Can wait until later rounds")
    print("💰 GOLD RINGS: Large rank disagreements (>20 spots) - opportunity for value")
    print("📊 FACETS: Separate panels for each position enable cleaner comparison")
    
except Exception as e:
    print(f"❌ Failed to create faceted plot: {str(e)}")
    print("Continuing with remaining visualizations...")

In [ ]:
def create_density_overview_plot(df):
    """Create overview plot with density for handling many players"""
    
    try:
        # Validate input data
        if df is None or df.empty:
            raise ValueError("Input DataFrame is empty or None")
            
        # Check for required columns
        required_columns = ['espn_rank', 'adp_rank', 'urgency_bin']
        missing_cols = [col for col in required_columns if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing required columns for density plot: {missing_cols}")
        
        # Create hexbin-style density plot
        fig = go.Figure()
        
        # Add density layer
        fig.add_trace(
            go.Histogram2d(
                x=df['espn_rank'],
                y=df['adp_rank'],
                nbinsx=30,
                nbinsy=30,
                colorscale='Blues',
                opacity=0.6,
                showscale=False,
                hoverinfo='skip'
            )
        )
        
        # Overlay urgent players only
        urgent_players = df[df['urgency_bin'] == 'Urgent']
        
        if not urgent_players.empty:
            # Build hover template dynamically based on available columns
            hover_template = ('<b>%{text}</b><br>')
            if 'position' in urgent_players.columns:
                hover_template += ('Position: ' + urgent_players['position'] + '<br>')
            hover_template += ('ESPN Rank: %{x}<br>' +
                             'ADP Rank: %{y}<br>')
            if 'vbd_score' in urgent_players.columns:
                hover_template += ('VBD Score: ' + urgent_players['vbd_score'].round(1).astype(str) + '<br>')
            if 'probability_available' in urgent_players.columns:
                hover_template += ('Availability: ' + (urgent_players['probability_available'] * 100).round(1).astype(str) + '%<br>')
            hover_template += '<extra></extra>'
            
            # Use marker_size if available, otherwise default
            marker_sizes = urgent_players['marker_size'] if 'marker_size' in urgent_players.columns else 12
            
            fig.add_trace(
                go.Scatter(
                    x=urgent_players['espn_rank'],
                    y=urgent_players['adp_rank'],
                    mode='markers+text',
                    marker=dict(
                        size=marker_sizes,
                        color='#DC143C',
                        opacity=0.9,
                        line=dict(width=1, color='white')
                    ),
                    text=urgent_players['player_name'],
                    textposition='top center',
                    textfont=dict(size=10, color='black'),
                    name='Urgent Players',
                    hovertemplate=hover_template
                )
            )
        
        # Add perfect agreement line
        max_rank = min(200, max(df['espn_rank'].max(), df['adp_rank'].max()))
        fig.add_trace(
            go.Scatter(
                x=[1, max_rank],
                y=[1, max_rank],
                mode='lines',
                line=dict(dash='dash', color='gray', width=2),
                name='Perfect Agreement',
                hoverinfo='skip'
            )
        )
        
        fig.update_layout(
            title='Draft Overview: Player Density with Urgent Targets Highlighted',
            xaxis_title='ESPN Rank (lower = better)',
            yaxis_title='ADP Rank (lower = better)',
            height=600,
            width=800,
            plot_bgcolor='white',
            font=dict(size=12)
        )
        
        fig.update_xaxes(range=[0, max_rank])
        fig.update_yaxes(range=[0, max_rank])
        
        return fig
        
    except Exception as e:
        print(f"❌ Error creating density plot: {str(e)}")
        raise

# Create density overview with error handling
try:
    density_plot = create_density_overview_plot(viz_data)
    density_plot.show()

    print("\n🗺️ DENSITY Overview:")
    print("📊 Blue density shows where most players cluster")
    print("🔴 Red markers highlight URGENT players only (draft immediately)")
    print("📝 Player names shown for urgent targets to reduce visual clutter")
    
except Exception as e:
    print(f"❌ Failed to create density plot: {str(e)}")
    print("Continuing with remaining visualizations...")

In [ ]:
def create_top_targets_table(df, n_players=20):
    """Create interactive table of top strategic targets"""
    
    try:
        # Validate input data
        if df is None or df.empty:
            raise ValueError("Input DataFrame is empty or None")
            
        # Check for required columns
        required_columns = ['player_name', 'urgency_bin']
        missing_cols = [col for col in required_columns if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing required columns for table: {missing_cols}")
        
        # Use urgency_rank if available, otherwise sort by probability_available
        if 'urgency_rank' in df.columns:
            top_targets = df.nsmallest(n_players, 'urgency_rank')
        elif 'probability_available' in df.columns:
            top_targets = df.nlargest(n_players, 'probability_available')
        else:
            # Fallback: just take first n_players
            top_targets = df.head(n_players)
        
        # Select available columns
        base_columns = ['player_name', 'urgency_bin']
        optional_columns = ['position', 'espn_rank', 'adp_rank', 'vbd_score', 
                           'probability_available', 'rank_divergence']
        
        available_columns = base_columns + [col for col in optional_columns 
                                          if col in top_targets.columns]
        
        top_targets = top_targets[available_columns].copy()
        
        # Format columns if they exist
        if 'probability_available' in top_targets.columns:
            top_targets['availability_pct'] = (top_targets['probability_available'] * 100).round(1)
        
        if 'vbd_score' in top_targets.columns:
            top_targets['vbd_score'] = top_targets['vbd_score'].round(1)
            
        if 'rank_divergence' in top_targets.columns:
            top_targets['rank_divergence'] = top_targets['rank_divergence'].round(1)
        
        # Create display columns mapping
        display_columns = {'player_name': 'Player', 'urgency_bin': 'Urgency'}
        if 'position' in top_targets.columns:
            display_columns['position'] = 'Pos'
        if 'espn_rank' in top_targets.columns:
            display_columns['espn_rank'] = 'ESPN'
        if 'adp_rank' in top_targets.columns:
            display_columns['adp_rank'] = 'ADP'
        if 'vbd_score' in top_targets.columns:
            display_columns['vbd_score'] = 'VBD'
        if 'availability_pct' in top_targets.columns:
            display_columns['availability_pct'] = 'Avail%'
        if 'rank_divergence' in top_targets.columns:
            display_columns['rank_divergence'] = 'Divergence'
        
        # Create the display dataframe
        display_df = top_targets.rename(columns=display_columns)
        
        print(f"\n🎯 TOP {min(n_players, len(display_df))} STRATEGIC TARGETS:")
        print("=" * 80)
        
        # Color-code by urgency
        for _, row in display_df.iterrows():
            urgency_emoji = {
                'Urgent': '🔴',
                'Decision': '🟠', 
                'Safe': '🟢',
                'Unknown': '⚪'
            }.get(row['Urgency'], '⚪')
            
            # Build display string dynamically
            display_str = f"{urgency_emoji} {row['Player']:<20}"
            
            if 'Pos' in row:
                display_str += f" {row['Pos']:<3}"
            if 'ESPN' in row:
                display_str += f" ESPN:{row['ESPN']:<3}"
            if 'ADP' in row:
                display_str += f" ADP:{row['ADP']:<3}"
            if 'VBD' in row:
                display_str += f" VBD:{row['VBD']:<5}"
            if 'Avail%' in row:
                display_str += f" Avail:{row['Avail%']:<5}%"
            if 'Divergence' in row:
                display_str += f" Div:{row['Divergence']:<5}"
            
            print(display_str)
        
        return display_df
        
    except Exception as e:
        print(f"❌ Error creating top targets table: {str(e)}")
        raise

# Create top targets table with error handling
try:
    top_targets_df = create_top_targets_table(viz_data, n_players=25)

    print("\n\n💡 STRATEGIC INSIGHTS:")
    print("🔴 URGENT players have <30% availability - draft immediately if available")
    print("🟠 DECISION players (30-80% available) - strong candidates for your next pick")
    print("🟢 SAFE players (>80% available) - can wait for later rounds")
    print("📊 High positive divergence = ESPN likes more than consensus (potential value)")
    print("📉 High negative divergence = ADP higher than ESPN suggests (potential reach)")
    
except Exception as e:
    print(f"❌ Failed to create top targets table: {str(e)}")
    print("Continuing with remaining analysis...")

In [ ]:
def create_positional_summary(df):
    """Create summary statistics by position and urgency"""
    
    try:
        # Validate input data
        if df is None or df.empty:
            raise ValueError("Input DataFrame is empty or None")
            
        # Check for required columns
        required_columns = ['position', 'urgency_bin', 'player_name']
        missing_cols = [col for col in required_columns if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing required columns for summary: {missing_cols}")
        
        # Build aggregation dictionary dynamically
        agg_dict = {'player_name': 'count'}
        
        if 'vbd_score' in df.columns:
            agg_dict['vbd_score'] = ['mean', 'max']
        if 'probability_available' in df.columns:
            agg_dict['probability_available'] = 'mean'
        if 'abs_divergence' in df.columns:
            agg_dict['abs_divergence'] = 'mean'
        
        summary = df.groupby(['position', 'urgency_bin']).agg(agg_dict).round(2)
        
        # Flatten column names
        if isinstance(summary.columns, pd.MultiIndex):
            summary.columns = ['_'.join(col).strip() if col[1] else col[0] 
                             for col in summary.columns.values]
        
        # Rename columns for clarity
        column_mapping = {
            'player_name_count': 'Count',
            'player_name': 'Count'
        }
        
        if 'vbd_score_mean' in summary.columns:
            column_mapping['vbd_score_mean'] = 'Avg_VBD'
        if 'vbd_score_max' in summary.columns:
            column_mapping['vbd_score_max'] = 'Max_VBD'
        if 'probability_available_mean' in summary.columns:
            column_mapping['probability_available_mean'] = 'Avg_Availability'
        if 'abs_divergence_mean' in summary.columns:
            column_mapping['abs_divergence_mean'] = 'Avg_Divergence'
        
        summary = summary.rename(columns=column_mapping)
        summary = summary.reset_index()
        
        print("\n📊 POSITIONAL BREAKDOWN:")
        print("=" * 70)
        
        # Get available positions
        available_positions = df['position'].unique()
        main_positions = ['RB', 'WR', 'QB', 'TE']
        positions_to_show = [pos for pos in main_positions if pos in available_positions]
        
        # Add any other positions not in main list
        other_positions = [pos for pos in available_positions if pos not in main_positions]
        positions_to_show.extend(sorted(other_positions))
        
        for pos in positions_to_show:
            pos_data = summary[summary['position'] == pos]
            if not pos_data.empty:
                print(f"\n{pos}:")
                for _, row in pos_data.iterrows():
                    urgency_emoji = {
                        'Urgent': '🔴',
                        'Decision': '🟠',
                        'Safe': '🟢',
                        'Unknown': '⚪'
                    }.get(row['urgency_bin'], '⚪')
                    
                    # Build display string dynamically
                    display_str = f"  {urgency_emoji} {row['urgency_bin']:<8}: {row['Count']:<3} players"
                    
                    if 'Avg_VBD' in row:
                        display_str += f", Avg VBD: {row['Avg_VBD']:<5}"
                    if 'Avg_Availability' in row:
                        display_str += f", Avg Avail: {row['Avg_Availability']*100:.1f}%"
                    
                    print(display_str)
        
        return summary
        
    except Exception as e:
        print(f"❌ Error creating positional summary: {str(e)}")
        raise

# Create positional summary with error handling
try:
    pos_summary = create_positional_summary(viz_data)

    print("\n\n🎯 KEY TAKEAWAYS:")
    print("• Focus on URGENT (🔴) players first - they may not be available later")
    print("• DECISION (🟠) players offer good value but require timing judgment") 
    print("• Use divergence to find potential steals (high ESPN rank, low ADP)")
    print("• Monitor availability percentages as the draft progresses")
    
    print("\n✅ NOTEBOOK EXECUTION COMPLETE!")
    print("All visualizations created with robust error handling.")
    print("If any errors occurred, check the data loading section and ensure")
    print("the main espn_probability_matrix.ipynb has been run successfully.")
    
except Exception as e:
    print(f"❌ Failed to create positional summary: {str(e)}")
    print("\nDespite this error, the main visualizations should still be available above.")